In [1]:
"""takes a CSV path, saves a new csv with duplicate players merged and fantasy points applied."""
# import dependancies
import pandas

In [2]:
# constants
# CSV is location of target csv to be altered and saved
CSV = "Offense_Only_list.csv"
with open(CSV, "r") as inbox:
    # DATA is the original data
    DATA = pandas.read_csv(inbox)
# for testing. will be removed from final version
# DATA = DATA.head(5)
# to test duplicate stuff
# DATA = DATA.append(DATA.head(10))
# LOCATION is save location/name
LOCATAION = "cleaned.csv"
# rename shorthand to more readable / convenient names
RENAME = {"P_Yards": "passing yards",
          "P_TD": "passing touchdowns", 
          "P_Int": "passing intercepetions", 
          "Rush_Yards": "rushing yards",
          "Rush_TD": "rushing touchdowns",
          "Fumbles": "fumbles",
          "Rec_Yards": "receiving yards",
          "Rec_TD": "receiving touchdowns",
          "Receptions": "receptions",
          "Kick_Returns": "kick returns",
          "Punt_Returns": "punt returns"
         }
# holds the fantasy values for each column header
END_POINTS = {}
ball = {
    "passing": {
        "yards": .04,
        "touchdowns": 4,
        "interceptions": -2
    },
    "rushing": {
        "yards": .1,
        "touchdowns": 6
    },
    "receiving": {
        "yards": .1,
        "touchdowns": 6,
        "receptions": 1
    }
}
for play in ball:
    for result in ball[play]:
        END_POINTS[f"{play} {result}"] = ball[play][result]
END_POINTS["fumbles"] = -2
for each in ["kick", "punt"]:
    END_POINTS[f"{each} returns"] = 6
# this solves a problem later easily
END_POINTS["Games_Played"] = 1

In [3]:
# functions

In [4]:
# rename headers
result = DATA.rename(columns = RENAME)
result
# drop the two beginning, unnamed columns. they were probably old indexes?
for each in [0, 1]:
    result.drop(result.columns[0], axis = 1, inplace = True)
# grab the duplicates
result["duplicate"] = result.duplicated(subset=["Name", "Master_Id"], keep=False)
duplicates = result.loc[result["duplicate"] == True]
# drop them from result.
result.drop_duplicates(["Name", "Master_Id"], keep = False, inplace = True)
# make a blank DF
corrected = pandas.DataFrame(columns = result.columns)
# for each duplicate:
for each in duplicates["Master_Id"].unique():
    # grab the loc
    ball = duplicates.loc[duplicates["Master_Id"] == each]
    # for each column
    for each in ball.columns:
        # if it's in our endpoints
        if each in END_POINTS:
            # sum it all up, multiply by endpoints
            ball.set_value(ball.index[0], each, ball[each].sum() * END_POINTS[each])
        # otherwise, it's good as-is
    # append to corrected
    corrected = corrected.append(ball.head(1))
# append the corrected df to result
result = result.append(corrected)
# drop the 'duplicate' field
result.drop("duplicate", axis = 1, inplace = True)
# save to location as a csv
with open(LOCATAION, "w") as outbox:
    result.to_csv(outbox, index = False)